In [ ]:
%pylab inline
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, ParticleFile, Variable, Field, BrownianMotion2D
from datetime import timedelta as delta
from datetime import datetime
import numpy as np
import math
from glob import glob

import xarray as xr
import matplotlib.pyplot as plt
import cartopy
import matplotlib.ticker as mticker

In [ ]:
class ParticleData(object):
    def __init__(self):
        self.id = []

def load_particles_file(fname, varnames):
    T = ParticleData()
    pfile = xr.open_dataset(str(fname), decode_cf=True)
    T.id = pfile.variables['trajectory'][:]

    for v in varnames:
        setattr(T, v,  np.ma.filled(pfile.variables[v][:], np.nan))
    pfile.close()
    
    return T

In [ ]:
from matplotlib import colors
pylab.rcParams['figure.figsize'] = (15, 17)

wstokes = [False, False, True]
wwind = [False, 0.01, 0.005]
axextent = [-280, -130, -45, 0]
dx = 0.5
startyears = range(2000, 2014)

projection = cartopy.crs.PlateCarree(central_longitude=-180)
fig, axes = plt.subplots(3, 2, subplot_kw={'projection': projection})
for i in range(len(wstokes)):
    withstokes = wstokes[i]
    withwind = wwind[i]
    ax = axes[i][:]

    bins = [np.arange(axextent[0], axextent[1], dx), np.arange(axextent[2], axextent[3], dx)]
    H = np.zeros((len(bins[0])-1, len(bins[1])-1))
    A = np.zeros((len(bins[0])-1, len(bins[1])-1))
    nA = np.zeros((len(bins[0])-1, len(bins[1])-1))
    for startyear in startyears:
        fname = 'pumiceevent_aug_%d' %startyear
        if withstokes:
            fname+='_wstokes'
        if withwind:
            fname+='_wind%.4d' % (withwind*1000)
        fname += '.nc'

        T = load_particles_file(fname, ['lon', 'lat', 'time'])
        T.fname = fname

        Htmp, xe, ye = np.histogram2d(T.lon[~np.isnan(T.lon)], T.lat[~np.isnan(T.lat)], bins=bins)
        H += Htmp/T.lon.shape[0]

        for d in range(T.lon.shape[0]):
            Atmp = np.zeros((len(bins[0])-1, len(bins[1])-1))
            for t in range(T.lon.shape[1]):
                xi = int((T.lon[d, t]-axextent[0]-dx/2)/dx)
                yi = int((T.lat[d, t]-axextent[2]-dx/2)/dx)
                if 0 <= xi < Atmp.shape[0] and 0 <= yi < Atmp.shape[1] and Atmp[xi, yi] == 0:
                    Atmp[xi, yi] = t
                    nA[xi, yi] += 1
            A+= Atmp
    A = A/nA

    xb = (xe[1:] + xe[:-1])/2
    yb = (ye[1:] + ye[:-1])/2

    for a in [0, 1]:
        ax[a].coastlines(resolution='50m')
        ax[a].add_feature(cartopy.feature.LAND)
        ax[a].set_extent(axextent, crs=cartopy.crs.PlateCarree())
        gl = ax[a].gridlines(crs=cartopy.crs.PlateCarree(central_longitude=-180), draw_labels=True,
                  linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
        gl.xlabels_top = False
        gl.ylabels_right = False
        gl.xformatter = cartopy.mpl.gridliner.LONGITUDE_FORMATTER
        gl.yformatter = cartopy.mpl.gridliner.LATITUDE_FORMATTER
        gl.xlocator = mticker.FixedLocator(np.arange(-360, 180, 10))
        gl.ylocator = mticker.FixedLocator(np.arange(-90, 90, 10))

    cmap = plt.get_cmap('Blues')

    pc = ax[0].pcolormesh(xb, yb, H.T, transform=cartopy.crs.PlateCarree(), norm=colors.LogNorm(), cmap=cmap)
    fig.colorbar(pc, ax=ax[0], orientation="horizontal")
    stokesstr = 'with stokes' if withstokes else 'without stokes'
    windstr = 'with %.3f windage' % withwind if withwind else 'without windage'
    ax[0].set_title('Particle density %s and %s ' %(windstr, stokesstr))

    ma = ax[1].pcolormesh(xb, yb, A.T, transform=cartopy.crs.PlateCarree())
    ax[1].set_title('Mean minimum particle age %s and %s [days]' %(windstr, stokesstr))
    fig.colorbar(ma, ax=ax[1], orientation="horizontal")

savefig('pumiceplot_climatologies_%s-%s_%s.pdf' % (startyears[0], startyears[-1], datetime.now().strftime("%Y%m%d_%H%M%S")))